## Классификация текстов: Spam or Ham

В этом задании на примере классического датасета Spambase Dataset (https://archive.ics.uci.edu/ml/datasets/spambase) мы попробуем сделать свой спам-фильтр c помощью библиотеки scikit-learn. Датасет содержит корпус текстов 5,574 смс с метками "spam" и "ham". 

### Данные

Для удобства данные приложены в описании задания

In [63]:
import pandas as pd
df = pd.read_csv('3_data.csv', encoding='latin-1')

Оставляем только интересующие нас колонки — тексты смс и метки:

In [64]:
df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Удаляем дублирующиеся тексты:

In [65]:
df = df.drop_duplicates('text')

Заменяем метки на бинарные:

In [66]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

### Предобработка текста (Задание 1)

Нужно дополнить функцию для предобработки сообщений, которая делает с текстом следующее:
* приводит текст к нижнему регистру;
* удаляет стоп-слова;
* удаляет знаки препинания;
* нормализует текст при помощи стеммера Snowball.

Предлагаем воспользоваться библиотекой nltk, чтобы не составлять список стоп-слов и не реализовывать алгоритм стемминга самостоятельно. Примеры использования стеммеров можно найти по ссылке (https://www.nltk.org/howto/stem.html).

In [67]:
from nltk import stem
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')

stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = [i for i in text.split() if not i in stopwords]
    text = [stemmer.stem(i) for i in text]
    text = " ".join([i for i in text])
    return text

# print(preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today."))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Проверка, что функция работает верно

In [68]:
assert preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.") == "im gonna home soon dont want talk stuff anymor tonight k ive cri enough today"
assert preprocess("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...") == "go jurong point crazi avail bugi n great world la e buffet cine got amor wat"

Применяем получившуюся функцию к текстам:

In [69]:
df['text'] = df['text'].apply(preprocess)

### Разделение данных на обучающую и тестовую выборки (Задание 2)

In [70]:
y = df['label'].values

Теперь нужно разделить данные на тестовую (test) и обучающую (train) выборку. В библиотеке scikit-learn для этого есть готовые инструменты.

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.25, random_state=94)

### Обучение классификатора (Задание 3)

Переходим к обучению классификатора.

Сначала извлекаем признаки из текстов. Рекомендуем попробовать разные способы и посмотреть, как это влияет на результат (подробнее о различных способах представления текстов можно прочитать по ссылке https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

Затем обучаем классификатор. Мы используем SVM, но можно поэкспериментировать с различными алгоритмами.

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# извлекаем признаки из текстов
vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [73]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

#обучаем подель SVM

model = LinearSVC(random_state = 94, C = 1.3)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

Для самопроверки. Если вы верно дополнили функцию ```preprocess```, то должны получиться следующие результаты оценки модели.

In [74]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.980     0.996     0.988      1133
           1      0.965     0.856     0.907       160

    accuracy                          0.978      1293
   macro avg      0.972     0.926     0.948      1293
weighted avg      0.978     0.978     0.978      1293



Выполним предсказание для конкретного текста

In [81]:
txt = "Take your prize, more than 100 computers, smartphones and TVs are supposed to be played in a free quiz. Call by phone 8 800 243 456"
txt = preprocess(txt)
txt = vectorizer.transform([txt])

In [82]:
model.predict(txt)

array([1])

In [62]:
Сообщение помечено как спам.

SyntaxError: ignored